In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
df = pd.read_csv("New folder/train.csv")
#df.columns = ['label', 'text']
df.head()

,sentiment,text
0,positive,@JetBlue @JayVig I like the inflight snacks! I...
1,positive,@VirginAmerica thanks guys! Sweet route over t...
2,negative,@USAirways Your exchange/credit policies are w...
3,negative,@USAirways but in the meantime I'll be sleepin...
4,negative,@VirginAmerica hold times at call center are a...


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [6]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=50

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
classes = df.sentiment.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{'positive': 0, 'negative': 1, 'neutral': 2}
{0: 'positive', 1: 'negative', 2: 'neutral'}


In [ ]:
df['label'] = df['sentiment'].apply(lambda x: class_tok2idx[x])
df.head()

,sentiment,text,label
0,positive,@JetBlue @JayVig I like the inflight snacks! I...,0
1,positive,@VirginAmerica thanks guys! Sweet route over t...,0
2,negative,@USAirways Your exchange/credit policies are w...,1
3,negative,@USAirways but in the meantime I'll be sleepin...,1
4,negative,@VirginAmerica hold times at call center are a...,1


In [ ]:
sequence_0 = "@united I will never fly with you again. Period."
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
model(**seq0_tokens).logits

number of tokens in seq0 is 14
{'input_ids': tensor([[  101,   137, 10280,   146,  1209,  1309,  4689,  1114,  1128,  1254,
           119, 16477,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[-0.5764,  0.0079,  0.2087]], grad_fn=<AddmmBackward0>)

In [ ]:
sequence_1 = "@united Delayed flight, but you did the best you could. Thank you united crew."
seq1_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq1_tokens['input_ids'].flatten())}")
model(**seq1_tokens).logits

number of tokens in seq1 is 22


tensor([[-0.6390, -0.0082,  0.2104]], grad_fn=<AddmmBackward0>)

In [ ]:
sequence_2 = "@united There is a reason why a only fly united, you're the best!"
seq2_tokens = tokenizer(sequence_2, return_tensors="pt")
print(f"number of tokens in seq2 is {len(seq2_tokens['input_ids'].flatten())}")
model(**seq2_tokens).logits

number of tokens in seq2 is 20


tensor([[-0.4897, -0.1721,  0.4280]], grad_fn=<AddmmBackward0>)

In [ ]:
ds_raw = Dataset.from_pandas(df[['label','text']])
ds_raw[0]

{'label': 0,
 'text': "@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat"}

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
ds[0]

{'label': 0,
 'text': "@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat",
 'input_ids': [101,
  137,
  13784,
  2064,
  19224,
  137,
  5525,
  2559,
  6512,
  146,
  1176,
  1103,
  1107,
  2087,
  4568,
  23029,
  1116,
  106,
  146,
  112,
  182,
  3754,
  1114,
  1128,
  3713,
  1113,
  123,
  120,
  1743,
  106,
  108,
  147,
  2559,
  10044,
  11220,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [ ]:
ds = ds.shuffle(seed=42)
ds[0]

{'label': 1,
 'text': '@AmericanAir 11 out of 11 delayed flights, you suck and getting worse',
 'input_ids': [101,
  137,
  1237,
  1592,
  3161,
  1429,
  1149,
  1104,
  1429,
  8088,
  7306,
  117,
  1128,
  13054,
  1105,
  2033,
  4146,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [ ]:
train_prop = 0.85
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))

In [ ]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 8500
len(ds_eval) = 1500


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=4, 
                                  do_train=True,
                                  report_to=None,
                                  output_dir="/kaggle/working",
                                  evaluation_strategy="steps",
                                  eval_steps=500,
                                  per_device_train_batch_size=10,
                                  per_device_eval_batch_size=10)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8500
  Num Epochs = 4
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization 

Step,Training Loss,Validation Loss,Accuracy
500,0.525700,0.506142,0.841333
1000,0.391800,0.474521,0.848000
1500,0.282900,0.497700,0.864667
2000,0.194900,0.689418,0.863333
2500,0.150500,0.653139,0.860000
3000,0.073300,0.735237,0.868667


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 10
Saving model checkpoint to /kaggle/working/checkpoint-500
Configuration saved in /kaggle/working/checkpoint-500/config.json
Model weights saved in /kaggle/working/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 10
Saving model checkpoint to /kaggle/working/checkpoint-1000
Configuration saved in /kaggle/working/checkpoint-1000/config.json
Model weights saved 

{'eval_loss': 0.7466463446617126,
 'eval_accuracy': 0.8693333333333333,
 'eval_runtime': 3.0487,
 'eval_samples_per_second': 492.021,
 'eval_steps_per_second': 49.202,
 'epoch': 4.0}

In [ ]:
df_test = pd.read_csv("/kaggle/input/cs39aa/test.csv")
df_test.head()

,id,text
0,0,@JetBlue okay awesome! Thank you!
1,1,@SouthwestAir wanting a ⭐️ for the ✈️ finding ...
2,2,"@united Funny thing happened, we arrived on ou..."
3,3,@united 2nd Late Flight flight today
4,4,@USAirways your service at PHL is abysmal. An ...


In [ ]:
ds_test_raw = Dataset.from_pandas(df_test)
ds_test_raw[0]

{'id': 0, 'text': '@JetBlue okay awesome! Thank you!'}

In [ ]:
ds_test = ds_test_raw.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
preds = trainer.predict(test_dataset=ds_test)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3000
  Batch size = 10


In [ ]:
test_preds = np.apply_along_axis(np.argmax, 1, preds.predictions) 
#ds_test['preds'] = pd.Series(test_preds.tolist())

In [ ]:
df_test['preds'] = test_preds.tolist()
df_test['sentiment'] = df_test['preds'].apply(lambda x: class_idx2tok[x])
df_test.head()

,id,text,preds,sentiment
0,0,@JetBlue okay awesome! Thank you!,0,positive
1,1,@SouthwestAir wanting a ⭐️ for the ✈️ finding ...,1,negative
2,2,"@united Funny thing happened, we arrived on ou...",2,neutral
3,3,@united 2nd Late Flight flight today,1,negative
4,4,@USAirways your service at PHL is abysmal. An ...,1,negative


In [ ]:
df_submission = df_test[['id','sentiment']]
df_submission.head()

,id,sentiment
0,0,positive
1,1,negative
2,2,neutral
3,3,negative
4,4,negative


In [ ]:
df_submission.to_csv('submission.csv', index=False)